<a href="https://colab.research.google.com/github/Maedeabm/Emotion-aware-Chatbot-Using-Fine-tuned-LLM/blob/main/Emotion_aware_Chatbot_Using_Fine_tuned_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A high-level implementation of creating an Emotion-aware Chatbot using a Large Language Model (LLM) project using Python and Google Colab.

* GPU support enabled (from the toolbar, Runtime -> Change runtime type -> Hardware accelerator -> GPU).

 1. Installing Necessary Libraries

In [1]:
!pip install transformers
!pip install torch



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.2 MB/s eta 0:00:00


In [2]:
import torch
print(torch.__version__)

2.0.1+cu118


2. Setup and Data Preparation:

2.1. Import Libraries

In [3]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader, random_split


2.2. Load Dataset from Kaggle

In [4]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d kaushik3497/friends-tv-series-screenplay-script
!unzip friends-tv-series-screenplay-script.zip

data = pd.read_csv("friends_quotes.csv")
print(data.head())  # This will help us see the first few rows and understand column names
print('data column is:', data.columns)  # This will give us the names of columns to confirm if 'Emotion' is there or not


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open friends-tv-series-screenplay-script.zip, friends-tv-series-screenplay-script.zip.zip or friends-tv-series-screenplay-script.zip.ZIP.
     author  episode_number           episode_title  \
0    Monica             1.0  Monica Gets A Roommate   
1      Joey             1.0  Monica Gets A Roommate   
2  Chandler             

2.3. Load and Preprocess Data

In [5]:
data = pd.read_csv("friends_quotes.csv")
data = data.dropna().reset_index(drop=True)


3. Dataset and Tokenizer

3.1. Initialize Tokenizer

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")


3.2. Custom Dataset

In [7]:

class FriendsDataset(Dataset):

    def __init__(self, tokenizer, dataframe, block_size):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attention_masks = []
        for dialogue in dataframe['quote']:
            encodings = tokenizer(dialogue, truncation=True, padding='max_length', max_length=block_size, return_tensors="pt")
            self.input_ids.append(encodings.input_ids)
            self.attention_masks.append(encodings.attention_mask)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {'input_ids': self.input_ids[idx], 'attention_mask': self.attention_masks[idx]}

4. Model and Training

4.1. Initialize Model

In [8]:
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")


4.2. Train Model

In [9]:
from torch.utils.data import DataLoader, Dataset

class TextDataset(Dataset):
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts)

    def __getitem__(self, idx):
        return torch.tensor(self.tokenized_texts[idx])

tokenizer.pad_token = tokenizer.eos_token
block_size = 128
dataset = FriendsDataset(tokenizer=tokenizer, dataframe=data, block_size=block_size)

dataloader = DataLoader(dataset, batch_size=4, shuffle=True)



Then, set up training parameters and train

In [10]:
first_batch = next(iter(dataloader))
print(first_batch.keys())


dict_keys(['input_ids', 'attention_mask'])


In [13]:
# 4.1. Initialize Model

model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 4.2. Train Model

# Set training parameters
EPOCHS = 3
BATCH_SIZE = 4
LEARNING_RATE = 2e-5
WARMUP_STEPS = 1e2
MAX_SEQ_LEN = 300  # or any value you deem fit

# Create DataLoader
dataset = FriendsDataset(tokenizer, data, MAX_SEQ_LEN)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, BATCH_SIZE, shuffle=False)

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=-1)

# Training loop
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    model.train()
    for batch in train_loader:
        # Move data to GPU if available
        input_ids = batch["input_ids"].squeeze().to(device)
        attention_mask = batch["attention_mask"].squeeze().to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        scheduler.step()

        print(f"Loss: {loss.item()}")

    # Validation loop
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].squeeze().to(device)
            attention_mask = batch["attention_mask"].squeeze().to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
            total_val_loss += outputs.loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss}")


OutOfMemoryError: ignored